# Download the dataset

In [1]:
# import kagglehub
#
# # Download latest version
# path = kagglehub.dataset_download("hojjatk/mnist-dataset")

# print("Path to dataset files:", path)

# Load the dataset

In [2]:
import numpy as np
import idx2numpy

train_images_file = "./data/train-images.idx3-ubyte"
train_labels_file = "./data/train-labels.idx1-ubyte"
test_images_file = "./data/t10k-images.idx3-ubyte"
test_labels_file = "./data/t10k-labels.idx1-ubyte"

X_train_raw = idx2numpy.convert_from_file(train_images_file)
y_train_raw = idx2numpy.convert_from_file(train_labels_file)
X_test_raw = idx2numpy.convert_from_file(test_images_file)
y_test_raw = idx2numpy.convert_from_file(test_labels_file)

### Convert (60000, 28, 28) to (60000, 784)

In [3]:
import pandas as pd

X_train = X_train_raw.reshape((X_train_raw.shape[0], -1)).T
y_train = y_train_raw.reshape((y_train_raw.shape[0], -1))
X_test = X_test_raw.reshape((X_test_raw.shape[0], -1)).T
y_test = y_test_raw.reshape((y_test_raw.shape[0], -1))

# X_train.shape >>> (60000, 784)

X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)
X_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)

In [4]:
X_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,59990,59991,59992,59993,59994,59995,59996,59997,59998,59999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_train_df.head()

,0
0,5
1,0
2,4
3,1
4,9


In [6]:
def init_params():
    W1 = np.random.randn(10, 784) * np.sqrt(2 / 784)
    W2 = np.random.randn(10, 10) * np.sqrt(2 / 10)
    b1 = np.zeros((10, 1))
    b2 = np.zeros((10, 1))

    return W1, b1, W2, b2

def relu(x):
    return np.maximum(x, 0)

def deriv_relu(Z):
    return Z > 0

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def stable_softmax(Z):
    Z_shifted = Z - np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z_shifted)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_propagation(W1, b1, W2, b2, X):
    print("forward propagation")
    Z1 = W1.dot(X) + b1
    A1 = relu(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = stable_softmax(Z2)

    print("Z1: ", Z1.shape)
    print("A1: ", A1.shape)
    print("Z2: ", Z2.shape)
    print("A2: ", A2.shape)

    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_propagation(Z1, A1, Z2, A2, W2, X, Y):
    print("back propagation")
    m = Y.size
    one_hot_Y = one_hot(Y)

    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * deriv_relu(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    print("dZ1: ", dZ1.shape)
    print("dW1: ", dW1.shape)
    print("db1: ", db1.shape)
    print("dZ2: ", dZ2.shape)
    print("dW2: ", dW2.shape)
    print("db2: ", db2.shape)

    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    print("update params")
    W1 = W1 - (alpha * dW1)
    b1 = b1 - (alpha * db1)
    W2 = W2 - (alpha * dW2)
    b2 = b2 - (alpha * db2)

    print("W1: ", W1.shape)
    print("b1: ", b1.shape)
    print("W2: ", W2.shape)
    print("b2: ", b2.shape)
    return W1, b1, W2, b2

In [7]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    Y = Y.flatten()
    return np.mean(predictions == Y)

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()

    for i in range(iterations):
        print("Iteration:", i)
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_propagation(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        print("Accuracy:", get_accuracy(get_predictions(A2), Y))
        print("--------------------")
    return W1, b1, W2, b2

In [8]:
print(X_train.shape)
print(y_train.shape)
print("----------------------------")
W1, b1, W2, b2  = gradient_descent(X_train, y_train, 500, 0.1)

(784, 60000)
(60000, 1)
----------------------------
Iteration: 0
forward propagation
Z1:  (10, 60000)
A1:  (10, 60000)
Z2:  (10, 60000)
A2:  (10, 60000)
back propagation
dZ1:  (10, 60000)
dW1:  (10, 784)
db1:  (10, 1)
dZ2:  (10, 60000)
dW2:  (10, 10)
db2:  (10, 1)
update params
W1:  (10, 784)
b1:  (10, 1)
W2:  (10, 10)
b2:  (10, 1)
Accuracy: 0.09395
--------------------
Iteration: 1
forward propagation
Z1:  (10, 60000)
A1:  (10, 60000)
Z2:  (10, 60000)
A2:  (10, 60000)
back propagation
dZ1:  (10, 60000)
dW1:  (10, 784)
db1:  (10, 1)
dZ2:  (10, 60000)
dW2:  (10, 10)
db2:  (10, 1)
update params
W1:  (10, 784)
b1:  (10, 1)
W2:  (10, 10)
b2:  (10, 1)
Accuracy: 0.10441666666666667
--------------------
Iteration: 2
forward propagation
Z1:  (10, 60000)
A1:  (10, 60000)
Z2:  (10, 60000)
A2:  (10, 60000)
back propagation
dZ1:  (10, 60000)
dW1:  (10, 784)
db1:  (10, 1)
dZ2:  (10, 60000)
dW2:  (10, 10)
db2:  (10, 1)
update params
W1:  (10, 784)
b1:  (10, 1)
W2:  (10, 10)
b2:  (10, 1)
Accuracy: 0

KeyboardInterrupt: 